# What is the best pokemon?

importing necessary libraries

In [3]:
import pandas as pd
import requests
import os

parse_json takes a string url as input and then returns a dict with info of the specific pokemon from the API

In [40]:
def parse_json(url, isInfo):
    response = requests.get(url)
    content = response.json()
    
    if isInfo:
        name = content['name']
        moveset = ', '.join([move['move']['name'] for move in content['moves']])
        types = ', '.join([p_type['type']['name'] for p_type in content['types']])
        stats = [stat['base_stat'] for stat in content['stats']]
        
        poke_info = {'NAME': name , 'TYPE': types, 'HP': stats[0], 'ATTACK': stats[1], 'DEFENSE': stats[2], 
                                    'SPECIAL-ATTACK': stats[3], 'SPECIAL-DEFENSE': stats[4], 'SPEED': stats[5], 
                                    'TOTAL': sum(stats), 'MOVESET': moveset}
    else:
        p_type_name = content['name']
        poke_info = {p_type_name: {relation: [name['name'] for name in content['damage_relations'][relation]] 
                                   for relation in content['damage_relations']}}
        

    return poke_info
        

run through every pokemon in the pokeDB API and then store them in a Dataframe called poke_df

In [4]:
#done through this method to lessen time to debug and not constantly deal with timeout on API

if not os.path.isfile('poke_db.xlsx'):
    poke_df = pd.DataFrame(columns=['NAME', 'TYPE', 'HP', 'ATTACK', 'DEFENSE', 
                                'SPECIAL-ATTACK', 'SPECIAL-DEFENSE', 'SPEED', 
                                'TOTAL', 'MOVESET'])

    poke_id = 1
    poke_id_secondary = 10001

    while True:
        try:
            pID = poke_id_secondary if poke_id > 1025 else poke_id
            poke_df = pd.concat([poke_df, pd.DataFrame([parse_json(f'https://pokeapi.co/api/v2/pokemon/{pID}/', True)])], ignore_index=True)

            if poke_id > 1025:
                poke_id_secondary += 1
            else:
                poke_id += 1
        except:
            break
        
    poke_df.to_excel('poke_db.xlsx')
else:
    poke_df = pd.read_excel('poke_db.xlsx')

get the type advantages and input them into a dictionary and excel file to handle simulation of matches

In [44]:
if not os.path.isfile('type_advantages.xlsx'):
    type_id = 1
    types = dict()
    
    while True:
        try:
            types.update(parse_json(f"https://pokeapi.co/api/v2/type/{type_id}", False))
            type_id += 1
        except:
            break
        
    move_adv_df = pd.DataFrame.from_dict(types)
    move_adv_df.to_excel('type_advantages.xlsx')
else:
    move_adv_df = pd.read_excel('type_advantages.xlsx')

In [43]:
move_adv_df

,normal,fighting,flying,poison,ground,rock,bug,ghost,steel,fire,water,grass,electric,psychic,ice,dragon,dark,fairy
double_damage_from,[fighting],"[flying, psychic, fairy]","[rock, electric, ice]","[ground, psychic]","[water, grass, ice]","[fighting, ground, steel, water, grass]","[flying, rock, fire]","[ghost, dark]","[fighting, ground, fire]","[ground, rock, water]","[grass, electric]","[flying, poison, bug, fire, ice]",[ground],"[bug, ghost, dark]","[fighting, rock, steel, fire]","[ice, dragon, fairy]","[fighting, bug, fairy]","[poison, steel]"
double_damage_to,[],"[normal, rock, steel, ice, dark]","[fighting, bug, grass]","[grass, fairy]","[poison, rock, steel, fire, electric]","[flying, bug, fire, ice]","[grass, psychic, dark]","[ghost, psychic]","[rock, ice, fairy]","[bug, steel, grass, ice]","[ground, rock, fire]","[ground, rock, water]","[flying, water]","[fighting, poison]","[flying, ground, grass, dragon]",[dragon],"[ghost, psychic]","[fighting, dragon, dark]"
half_damage_from,[],"[rock, bug, dark]","[fighting, bug, grass]","[fighting, poison, bug, grass, fairy]","[poison, rock]","[normal, flying, poison, fire]","[fighting, ground, grass]","[poison, bug]","[normal, flying, rock, bug, steel, grass, psyc...","[bug, steel, fire, grass, ice, fairy]","[steel, fire, water, ice]","[ground, water, grass, electric]","[flying, steel, electric]","[fighting, psychic]",[ice],"[fire, water, grass, electric]","[ghost, dark]","[fighting, bug, dark]"
half_damage_to,"[rock, steel]","[flying, poison, bug, psychic, fairy]","[rock, steel, electric]","[poison, ground, rock, ghost]","[bug, grass]","[fighting, ground, steel]","[fighting, flying, poison, ghost, steel, fire,...",[dark],"[steel, fire, water, electric]","[rock, fire, water, dragon]","[water, grass, dragon]","[flying, poison, bug, steel, fire, grass, dragon]","[grass, electric, dragon]","[steel, psychic]","[steel, fire, water, ice]",[steel],"[fighting, dark, fairy]","[poison, steel, fire]"
no_damage_from,[ghost],[],[ground],[],[electric],[],[],"[normal, fighting]",[poison],[],[],[],[],[],[],[],[psychic],[dragon]
no_damage_to,[ghost],[ghost],[],[steel],[flying],[],[],[normal],[],[],[],[],[ground],[dark],[],[fairy],[],[]
